<a href="https://colab.research.google.com/github/SriDivya3009/Fmml/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

#1ANSWER

The accuracy of the validation set can be affected when you increase or decrease the percentage of data allocated to the validation set during the training of machine learning or deep learning models. The validation set is an essential part of model evaluation and hyperparameter tuning. Here's what happens when you adjust the percentage of the validation set:

Increase the Percentage of Validation Set:

Pros:

More data for validation means a better estimate of how well your model generalizes to unseen data. This can result in a more reliable evaluation of your model's performance.
It can help identify overfitting more effectively because the model has less training data, which can make overfitting more apparent.
Cons:

Less data for training, which can lead to slower model convergence and potentially lower training accuracy.
In some cases, if you allocate too much data to the validation set, your model might not have enough data to learn meaningful patterns during training.
Decrease the Percentage of Validation Set:

Pros:

More data for training, which can help the model converge faster and potentially achieve higher training accuracy.
If you have a limited dataset, a smaller validation set can help you utilize more data for training.
Cons:

The evaluation on a smaller validation set may not be as reliable. It might not effectively capture the model's generalization performance, and the estimated accuracy might be more variable.
The choice of the percentage allocated to the validation set is a trade-off between having a reliable estimate of model performance and ensuring that the model has enough data to learn meaningful patterns without overfitting. There is no one-size-fits-all answer, and the optimal percentage often depends on the specific dataset and problem you are working on.

In practice, a common split ratio for dataset partitioning is 70-80% for training and 20-30% for validation or testing. However, these percentages can vary based on the size of your dataset, the complexity of your model, and the goals of your project. It's often a good practice to experiment with different validation set percentages to find the right balance for your specific task. Cross-validation techniques can also be useful in cases where you have limited data, as they provide a more robust evaluation by repeatedly partitioning the data into training and validation subsets.


#2ANSWER


The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. Here's how:

Underfitting and Overfitting: The size of the training set is crucial in determining how well your model can learn the underlying patterns in the data. If your training set is too small, your model might underfit, meaning it doesn't capture the complexity of the data. Conversely, if your training set is too large, it's possible to overfit, where the model memorizes the training data but fails to generalize to unseen data. This can impact the model's performance on both the validation and test sets.

Validation Set Size: The size of the validation set is essential for estimating how well your model generalizes to new, unseen data. A smaller validation set may lead to noisier estimates of model performance because it has less data to evaluate the model's performance reliably. Conversely, a larger validation set can provide a more accurate estimate of model performance.

Generalization: The validation set helps you tune hyperparameters and make decisions about the model's architecture. If your validation set is too small, it might not represent the full range of variability in the data, leading to suboptimal hyperparameter choices and model selection.

Predicting Test Set Accuracy: The validation set is often used to fine-tune the model and select the best-performing version of the model. If the validation set is a good representation of the test set in terms of data distribution and size, it can provide a reasonably accurate estimate of how well the model will perform on unseen data. However, if the validation set is too different from the test set, the prediction of test set accuracy might not be accurate.

In summary, the size of the training and validation sets plays a crucial role in the model's ability to learn from the data, generalize to unseen data, and accurately predict performance on the test set. It's essential to strike a balance between the sizes of these sets and ensure they are representative of the data distribution to make reliable predictions about test set accuracy. Cross-validation techniques can also be helpful when you have limited data to better estimate model performance.


#3ANSWER

The choice of the percentage to reserve for the validation set can depend on several factors, including the size of your overall dataset, the complexity of your model, and your specific goals. However, a commonly used split ratio is 70-30 or 80-20 for training-validation, respectively. In other words, you would use 70% or 80% of your data for training and the remaining 30% or 20% for validation. Here's a breakdown of these two common choices:

1. *70-30 Split (Training-Validation)*:
   - *Advantages*: This split provides a larger portion of the data for training, which can be beneficial if you have a substantial dataset. More training data can help your model learn better and potentially capture more complex patterns.
   - *Considerations*: The validation set is smaller in this case, which may result in slightly noisier estimates of model performance, especially if your dataset is small or highly variable.

2. *80-20 Split (Training-Validation)*:
   - *Advantages*: A larger validation set (20%) can provide a more reliable estimate of your model's performance since it has more data to evaluate the model's generalization. This can be especially useful when you have a smaller dataset.
   - *Considerations*: With an 80-20 split, you have slightly less data for training, which can be a concern if your dataset is already limited in size or if your model is very complex and requires more training data.

Ultimately, there's no one-size-fits-all answer, and the split ratio should be chosen based on your specific circumstances. Here are some considerations to keep in mind:

- *Dataset Size*: If you have a large dataset, you can allocate a smaller percentage for validation (e.g., 20-30%) because the absolute number of samples in the validation set will still be reasonably large.

- *Model Complexity*: More complex models often require larger training datasets to avoid overfitting. If you're using a simple model, you may be able to get away with a smaller validation set.

- *Cross-Validation*: If you have limited data or want a more robust estimate of model performance, you can consider techniques like k-fold cross-validation, where the data is divided into k subsets, and each fold serves as both a training and validation set in turn.

- *Domain Knowledge*: Consider any domain-specific knowledge or prior experience you have with similar datasets. This can guide your decision on how to split the data.

In practice, it's common to start with a 70-30 or 80-20 split and then adjust as needed based on experimentation and the specific characteristics of your data and modeling goals.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#1ANSWER

Yes, averaging the validation accuracy across multiple splits can indeed provide more consistent and robust results when assessing the performance of a machine learning model. This technique is known as cross-validation, and it is widely used in the machine learning community to get a more reliable estimate of a model's performance. Here's how it works:

1. *K-Fold Cross-Validation*: The most common form of cross-validation is K-fold cross-validation. In this approach, the dataset is divided into K subsets or "folds" of roughly equal size. The model is trained and evaluated K times, each time using a different fold as the validation set and the remaining data as the training set.

2. *Consistency*: Cross-validation helps to mitigate the impact of random variability in the data and provides a more stable estimate of the model's performance. By averaging the validation scores across multiple folds, you reduce the influence of any single random split, making your evaluation more robust.

3. *Bias-Variance Tradeoff*: Cross-validation can also help you assess the bias-variance tradeoff of your model. If your model consistently performs well across all folds, it suggests that the model has a good balance between bias and variance. On the other hand, if there is a significant variability in performance, it might indicate that the model is overfitting or underfitting in some cases.

4. *Hyperparameter Tuning*: Cross-validation is particularly useful when tuning hyperparameters (e.g., learning rate, regularization strength). You can search for the hyperparameters that yield the best average validation performance across multiple folds, helping you avoid overfitting to a specific validation set.

5. *More Efficient Use of Data*: Cross-validation allows you to make better use of your available data. In a single train-validation split, you might be wasting a significant portion of your data for validation, especially when your dataset is small. Cross-validation ensures that all data points are used for both training and validation at some point.

Common values for K in K-fold cross-validation are 5 and 10, but you can choose an appropriate value based on your dataset size and computational resources. Leave-One-Out Cross-Validation (LOOCV), where each data point is a separate fold, is also an option for small datasets but can be computationally expensive.

In summary, cross-validation is a valuable technique to obtain more consistent and reliable estimates of a model's performance, and it is often used in practice to assess machine learning models and make informed decisions about model selection and hyperparameter tuning.


#2ANSWER


Cross-validation, while providing a more robust estimate of a model's performance compared to a single train-validation split, still does not directly estimate test accuracy. Instead, it provides a more accurate estimate of how well your model is likely to perform on unseen data based on the data you have. Here's why:

1. *Estimate of Model Performance*: Cross-validation gives you a better estimate of how well your model generalizes to new, unseen data compared to a single train-validation split. By averaging the results across multiple folds, you reduce the impact of randomness and variability in the data, providing a more reliable evaluation of your model's performance on the data you have.

2. *Performance on Test Data*: The true test of a model's accuracy is its performance on a completely independent test dataset that it has never seen during training or validation. Cross-validation doesn't replace this step; it serves as a proxy to help you make informed decisions about your model before deploying it in a real-world scenario.

3. *Validation vs. Test Set*: The validation set is used to make decisions about model selection, hyperparameter tuning, and general model assessment during the development phase. It's crucial for guiding the model's development but should not be confused with the test set, which serves as an unbiased evaluation of the final model.

4. *Data Distribution*: Cross-validation assumes that the data distribution across folds is representative of the overall dataset. If your data is not randomly sampled or has specific patterns, cross-validation results might not fully reflect the model's performance on your test data.

To obtain an accurate estimate of test accuracy, you should reserve a portion of your data as a true test set that is not used during model development or hyperparameter tuning. This test set should be kept entirely separate until the final evaluation stage. The test accuracy on this independent test set will provide the most accurate estimate of how your model performs in real-world scenarios.

In summary, while cross-validation provides a more robust estimate of your model's performance on your available data, it doesn't directly estimate test accuracy. Test accuracy is only determined by evaluating your model on a completely independent test dataset that has not been used in any way during model development or validation.


#3ANSWER

The number of iterations or epochs in training a machine learning model can have an impact on the estimate of the model's performance, but it does not necessarily guarantee a better estimate with higher iterations. The effect of the number of iterations depends on various factors, and there is a trade-off to consider.

Here are some key points to understand:

1. *Underfitting and Overfitting*: The number of iterations can affect the model's fit to the training data. With too few iterations, the model may not have enough training to learn the underlying patterns in the data, resulting in underfitting. On the other hand, with too many iterations, the model may start to memorize the training data, leading to overfitting, where it performs well on the training data but poorly on unseen data.

2. *Validation Performance*: During training, it's common to monitor the model's performance on a validation set (e.g., using a validation dataset or cross-validation). The validation performance may initially improve with more iterations as the model learns, but it can eventually plateau or even degrade if overfitting occurs. The point at which the validation performance plateaus or starts to degrade can vary depending on the dataset and model complexity.

3. *Early Stopping*: To find an optimal balance, many practitioners use a technique called early stopping. This involves monitoring the validation performance during training and stopping the training process when the validation performance no longer improves or starts to degrade. This prevents the model from overfitting and can lead to better generalization.

4. *Computational Resources*: The number of iterations also affects the computational resources required for training. More iterations typically require more time and computing power. Therefore, you need to consider practical constraints when choosing the number of iterations.

In summary, there is no universal rule that more iterations always lead to a better estimate of a model's performance. The appropriate number of iterations depends on your specific dataset, model architecture, and regularization techniques. It's important to strike a balance between providing the model with enough training to learn meaningful patterns and avoiding overfitting. Monitoring validation performance and using techniques like early stopping can help you determine when to stop training and obtain a good estimate of your model's performance.


#4ANSWER

Increasing the number of iterations can help to some extent when dealing with a very small training dataset or validation dataset, but it may not fully compensate for the limitations of small data. Here are some considerations:

1. Overfitting: Increasing the number of iterations on a small training dataset can increase the risk of overfitting. The model may start to memorize the limited training data rather than learning generalizable patterns. This can result in a model that performs very well on the training data but poorly on unseen data.

2. Limited Diversity: Small datasets often lack diversity, and the model may not encounter a wide range of examples. Increasing iterations won't magically introduce more diversity into the data. If the data is not representative of the problem space, the model's performance may still be limited.

3. Validation Set: If your validation dataset is also small, increasing iterations may not significantly improve the reliability of the estimate of your model's performance. Validation set size is crucial for assessing model generalization, and a small validation set can lead to noisy or unreliable performance estimates.

4. Data Augmentation: Instead of solely relying on increasing iterations, you can also consider data augmentation techniques. Data augmentation involves creating new training examples by applying various transformations (e.g., rotation, translation, cropping) to the existing data. This can effectively increase the effective size of your training dataset and help the model learn more robust features.

5. Regularization: Applying regularization techniques (e.g., L1, L2 regularization, dropout) can help mitigate overfitting, especially when dealing with small datasets. These techniques encourage the model to be less reliant on individual data points and focus on learning more general patterns.

6. Transfer Learning: If applicable, consider using pre-trained models and fine-tuning them on your small dataset. Transfer learning allows you to leverage knowledge learned from larger, related datasets and adapt it to your specific problem.

In summary, while increasing the number of iterations can provide some benefit, it's important to be cautious about overfitting and the limitations of small datasets. A more comprehensive approach involves a combination of techniques, including data augmentation, regularization, transfer learning, and careful model selection, to make the best use of limited data.